In [ ]:
###TODO dqn, change TOTAL_EXPLORATION_STEPS to NUM_EXPLORATION_STEPS
###TODO DQN, could vary EPSILON_RANGE as well
###TODO DQN render=False default argument for functions
###TODO DQN model.fit nb_epoch=10 default?
### t vs frame in comments

In [56]:
import numpy as np
import gym
from keras.engine import training
from keras.models import Sequential
from keras.models import model_from_json
from keras.layers import Dense, Activation
import tensorflow as tf
from collections import deque
from sys import stdout
import json
from os import path, makedirs

In [57]:
np.random.seed(3293734)

In [58]:
ENV = gym.make("LunarLander-v2")
NUM_INPUT = ENV.reset().shape[0]
NUM_ACTION = ENV.action_space.n
ACTIONS = np.arange(0, ACTION_DIM)

[2016-10-28 23:38:12,714] Making new env: LunarLander-v2


In [59]:
NUM_EPISODE = 1000
GAMMA = 0.99
FRAME_MEM = 1
HIDDEN_DIM = 200
ACTIVATION_VAL = 'tanh'
INITIALIZATION_VAL = 'glorot_uniform'
ACTIVATION_POL ='relu'
INITIALIZATION_POL = 'glorot_normal'

In [60]:
def _create_network(model_type): 
    model = Sequential()
    
    if model_type == 'value':
        # Regression function estimate for calculating the advantage with advantage actor-critic policy gradients
        model.add(Dense(HIDDEN_DIM, init=INITIALIZATION_VAL, input_dim=FRAME_MEM*NUM_INPUT)) 
        model.add(Activation(ACTIVATION_VAL))
        model.add(Dense(1, init=INITIALIZATION_VAL))
        model.compile(optimizer='rmsprop', loss='mse') ###TODO regression fchollet bookmarks, add to biblio
    
    if model_type == 'policy':
        model.add(Dense(HIDDEN_DIM, init=INITIALIZATION_POL, input_dim=FRAME_MEM*NUM_INPUT)) 
        model.add(Activation(ACTIVATION_POL))
        model.add(Dense(NUM_ACTION, init=INITIALIZATION_POL))
        model.add(Activation('softmax'))
        model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [61]:
# https://github.com/fchollet/keras/issues/3062 ###TODO in biblio
def get_trainable_params(model):
    params = []
    for layer in model.layers:
        params += training.collect_trainable_weights(layer)
    return params

In [72]:
def advantage_actor_critic(model_val, model_pol, render=False): 
    # Start a new epoch
    t = 0; success = 0; solved = False

    # Play through episodes
    for episode in range(NUM_EPISODE+1): 
        te = 0
        trpre = []
        x_t = ENV.reset()
        s_t = np.tile(x_t, FRAME_MEM)
        done = False

        # Start an episode
        while not done:
            t += 1; te += 1
            
            # Reset the environment for a new gaming step
            if render: ENV.render()
            
            # POLICY ESTIMATOR: PREDICTION
            # Estimate probabilities for each possible action
            probs = model_pol.predict(s_t[np.newaxis])[0]
            
            # Take an action: Sample an action from the probabilities distribution
            a_t = np.random.choice(ACTIONS, p=probs)
            
            # Observe after action a_t
            s_t1, r_t, done, info = ENV.step(a_t)
            
            # Keep track of the episode transition and the probability of the action taken
            trpre.append((s_t, a_t, r_t, probs[a_t]))
            
            # Update state
            s_t = s_t1
        
        # Update policy after each episode 
        for t, transition in enumerate(trpre):
        
            # Calculate the total discounted reward after this episode frame
            total_return = sum(GAMMA**i * j[2] for i, j in enumerate(trpre[t:])) 

            # Get state at t
            ep_s_t = transition[0][np.newaxis]

            # Update value estimator
            model_val.fit(ep_s_t, np.asarray([total_return]), verbose=0)

            # Calculate baseline for the action taken / Predict the baseline
            baseline_value = model_val.predict(ep_s_t)[0][0]

            # Calculate advantage FOR THE PICKED ACTION ONLY (target / y_true)
            advantage = total_return - baseline_value

            # Update policy estimator / Compute the policy gradients
            # loss = (-ln(prob of action taken) * advantage/target
            # (cannot use standard model.fit keras)
            # https://github.com/fchollet/keras/issues/3062 in references on top ###TODO
            loss = -np.log(transition[3]) * advantage
            network_params = get_trainable_params(model_pol)
            param_grad = tf.gradients(loss, network_params)

In [ ]:
model_pol, model_val = _create_network("policy"), _create_network("value")
advantage_actor_critic(model_val, model_pol) # Train model / Play episode